In [10]:
%config IPCompleter.greedy=True
import os.path
import csv
import sys
from sqlalchemy import create_engine
csv.field_size_limit(sys.maxsize)
import petl as etl
from collections import OrderedDict
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient, BlobBlock

In [11]:
account_name='datalake2dictas'
account_key='wlxJHcWvtVhPpL/zs6l+F1bJGKZnJ4HppZcVyh+ns32oH46E3dY/HBLau3V6um9hv+KZf/3mXEAL5nHD41X3jg=='
container_from = 'transient'
container_to = 'raw'
upload_file = '/opt/jupyter_etl_dictas/datalake/transient/beneficiario/beneficiarios_atualizado.csv'
_LOCALDIR = '/opt/jupyter_etl_dictas/datalake/transient'

In [12]:
cc_transient = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name=container_from, credential=account_key)
cc_raw = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name=container_to, credential=account_key)
cc_refined = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name='refined', credential=account_key)

In [15]:
_TEMP_FILE='datalake/temp'
_METADADOS='metadado/list.csv'
_SOURCE='17'
_PROC_FILES='datalake/proc'
_NM_ARQ = 'EMPRESA.csv'

### Carrega blob Vidas

#### Verifica Arquivo no Container

In [14]:
blob_list = cc_transient.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')
    

BENEFICIARIO.csv

CONSULTAS.csv

EMPRESA.csv

PRESTADORES.csv

RECEITAS.csv

SERVICOS.csv



#### Faz Download para diretório local

In [16]:
with open (f'{_PROC_FILES}/{_SOURCE}/{_NM_ARQ}', 'wb') as data_from :
    data_from.write(cc_refined.get_blob_client(_NM_ARQ).download_blob ().readall ())

#### Carrega dados para transformação

In [17]:
table = etl.fromcsv(f'{_PROC_FILES}/{_SOURCE}/{_NM_ARQ}', delimiter='|')
table

pk_empresa,nome,razao_social,cnpj,fk_operadora,dt_proc
395988000135,ITAIPU,ITAIPU,00395988000135,17,13/3/2020
469585000193,FACEB,FACEB,00469585000193,17,13/3/2020
2341467000120,AMAZONAS ENERGIA,AMAZONAS ENERGIA,02341467000120,17,13/3/2020
2983428000127,CEMIG TELECOM,CEMIG TELECOM,02983428000127,17,13/3/2020
3475839000174,ROSAL ENERGIA,ROSAL ENERGIA,03475839000174,17,13/3/2020


In [15]:
#Transforma sexo e tira os espaços vazios e substitui string 'NULL' por ''
def rowmapper(row):
    transatv = {'NÃO ATIVO': 'I', 'ATIVO': 'A'}
    return [row[0].strip(),
            row[1].strip(),
            #row[2].strip()
            transatv[row['ativo'].strip()] if row['ativo'].strip() in transatv else None
            ]

table1 = etl.rowmap(table, rowmapper,
                    header=['mes_competencia','fk_beneficiario','ativo'
                           ])
table1

mes_competencia,fk_beneficiario,ativo
022017,00000001019011,A
022017,00000005000003,A
022017,00000008001570,I
022017,00000009000004,A
022017,00000011000004,A


#### Add fk_empresa

In [18]:
table2 = etl.addfields(table,[('cd_empresa',_SOURCE)])
#table2
#etl.tocsv(table2, f'{_PROC_FILES}/{_SOURCE}/t{_NM_ARQ}', delimiter='|')

etl.nrows(table)

21

In [16]:
upload_file=f'{_PROC_FILES}/{_SOURCE}/t{_NM_ARQ}'
#print(upload_file)
try:
    with open (upload_file, "rb") as data :
        #print(data)
        cc_raw.upload_blob(_NM_ARQ, data, overwrite = True)
finally:
    print("Tudo carregado")

Tudo carregado


#### Verifica se arquivo está no Blob

In [17]:
blob_list = cc_raw.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')

BENEFICIARIO.csv

CONSULTAS.csv

CUSTOS.csv

DIARIAS.csv

EMPRESA.csv

PRESTADORES.csv

RECEITAS.csv

SERVICOS.csv

VIDAS.csv



In [ ]:
def line_count(fname):
    return sum(1 for line in open(fname))

In [7]:
engine = create_engine(
    "postgresql+psycopg2://giovani.stefani@prd-dictas-postgresql-server.postgres.database.azure.com:B7hqx9mFi19J8BTE@prd-dictas-postgresql-server.postgres.database.azure.com:5432/dictas")

In [8]:
etl.todb(table, engine, 'vidas')l

KeyboardInterrupt: 